In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install sentence-transformers
!pip install mysql-connector-python
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=118fe39177474c1c3c8a9e8e297858fd2d83d19e79e7a2dc2309c4d524da78ea
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all 

In [ ]:
from fastapi import FastAPI, HTTPException, Query
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import nest_asyncio
import uvicorn
import requests
import mysql.connector

from sentence_transformers import SentenceTransformer, util
from pyngrok import ngrok

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Initialize the Sentence Transformer model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

db = mysql.connector.connect(
    host="ayurconnect.mysql.database.azure.com",
    user="service_admin",
    password="123.Abc.",
    database="ayurconnect",
)

@app.get("/search/")
async def search_top_results(query: str = Query(..., description="Search text")):
    try:
        # Create a cursor for the database connection
        cursor = db.cursor()

        # Execute a SQL query to fetch all sentences from the database
        cursor.execute("SELECT c.Id, c.Header, c.Body, r.Body FROM contents c LEFT OUTER JOIN responses r ON c.Id = r.ContentId WHERE c.IsDeleted = 0;")

        # Fetch all sentences and their IDs from the database
        corpus_data = [(row[0], row[1], row[2]) for row in cursor.fetchall()]

        # Extract the sentences from the data
        corpus = [f"{row[1]} {row[2]}" for row in corpus_data]

        # Encode the corpus sentences
        corpus_embeddings = embedder.encode(corpus, show_progress_bar=True, convert_to_tensor=True)

        # Encode the query sentence
        query_embedding = embedder.encode(query, show_progress_bar=True, convert_to_tensor=True)

        # Calculate cosine similarity
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]

        # Get the top 5 most similar sentences
        top_k = min(5, len(corpus))
        top_results = sorted(enumerate(cos_scores), key=lambda x: x[1], reverse=True)[
            :top_k
        ]

        # Construct the Get Content by Id API endpoint
        api_url = "https://ayurconnect-service-1.azurewebsites.net/api/content/{}"

        # Initialize an empty list to store the results
        results = []

        # Loop through the top_results and make API requests for each ID
        for idx, _ in top_results:
            api_request_url = api_url.format(corpus_data[idx][0])
            response = requests.get(api_request_url)

            if response.status_code == 200:
                result_data = response.json()
                print(result_data)
                results.append(result_data)
            else:
                print(f"Failed to fetch data for ID {corpus_data[idx][0]}")

        return JSONResponse(content=results, media_type="application/json")
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [920]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://adc2-34-170-233-85.ngrok.io


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'header': 'What are the signs and symptoms that indicate the possibility of dengue fever?', 'body': 'When should I suspect that I have dengue fever?', 'contentType': 'QUESTION', 'responses': [], 'id': 4, 'userId': '64e8d03eb61479919c13c218', 'createdDate': '2023-10-28T09:34:10.415354', 'updatedDate': '2023-10-28T09:34:10.415354', 'isDeleted': False}
{'header': 'What are the initial symptoms of dengue fever?', 'body': 'High fever (40°C/104°F) accompanied by severe headache, pain behind the eyes, muscle and joint pains, nausea, vomiting, swollen glands, and rash.', 'contentType': 'ARTICLE', 'responses': [], 'id': 3, 'userId': '64e8d03eb61479919c13c218', 'createdDate': '2023-10-28T09:31:44.38449', 'updatedDate': '2023-10-28T09:31:44.38449', 'isDeleted': False}
{'header': 'Can consuming rice result in an increase in blood sugar levels?', 'body': "People with diabetes who are striving for a well-balanced diet may have doubts about whether carbohydrate-rich foods such as white rice should b

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [920]
